In [18]:
import pandas as pd
import sklearn as sk
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
import tensorflow as tf
import requests
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from sklearn import preprocessing

In [19]:
data = pd.read_csv(r"C:\Users\User\Desktop\Python\data.csv")
data

,matchid,timestamp,game_time,game_state,pick_1,pick_2,pick_3,pick_4,pick_5,pick_6,pick_7,pick_8,pick_9,pick_10,r1_hero_id,r1_level,r1_kills,r1_deaths,r1_assists,r1_denies,r1_lh,r1_gold,r1_x,r1_y,r1_net_worth,r2_hero_id,r2_level,r2_kills,r2_deaths,r2_assists,r2_denies,r2_lh,r2_gold,r2_x,r2_y,r2_net_worth,r3_hero_id,r3_level,r3_kills,r3_deaths,r3_assists,r3_denies,r3_lh,r3_gold,r3_x,r3_y,r3_net_worth,r4_hero_id,r4_level,r4_kills,r4_deaths,r4_assists,r4_denies,r4_lh,r4_gold,r4_x,r4_y,r4_net_worth,r5_hero_id,r5_level,r5_kills,r5_deaths,r5_assists,r5_denies,r5_lh,r5_gold,r5_x,r5_y,r5_net_worth,d1_hero_id,d1_level,d1_kills,d1_deaths,d1_assists,d1_denies,d1_lh,d1_gold,d1_x,d1_y,d1_net_worth,d2_hero_id,d2_level,d2_kills,d2_deaths,d2_assists,d2_denies,d2_lh,d2_gold,d2_x,d2_y,d2_net_worth,d3_hero_id,d3_level,d3_kills,d3_deaths,d3_assists,d3_denies,d3_lh,d3_gold,d3_x,d3_y,d3_net_worth,d4_hero_id,d4_level,d4_kills,d4_deaths,d4_assists,d4_denies,d4_lh,d4_gold,d4_x,d4_y,d4_net_worth,d5_hero_id,d5_level,d5_kills,d5_deaths,d5_assists,d5_denies,d5_lh,d5_gold,d5_x,d5_y,d5_net_worth,r_l1_t1,r_l1_t2,r_l1_t3,r_l2_t1,r_l2_t2,r_l2_t3,r_l3_t1,r_l3_t2,r_l3_t3,r_l2_t4_1,r_l2_t4_2,r_l1_b1,r_l1_b2,r_l2_b1,r_l2_b2,r_l3_b1,r_l3_b2,d_l1_t1,d_l1_t2,d_l1_t3,d_l2_t1,d_l2_t2,d_l2_t3,d_l3_t1,d_l3_t2,d_l3_t3,d_l2_t4_1,d_l2_t4_2,d_l1_b1,d_l1_b2,d_l2_b1,d_l2_b2,d_l3_b1,d_l3_b2
0,5847436819,1819,896,5,58,102,31,32,53,108,101,12,126,73,102,6,0,5,4,1,9,245,-0.066197,-0.135855,2035,101,7,3,4,3,0,6,182,-0.251940,-0.172926,2687,73,10,0,2,0,6,92,715,-0.263461,0.216023,6815,53,9,3,4,4,11,65,1066,0.166145,-0.385070,5071,32,9,3,1,2,8,42,657,-0.114449,-0.242509,4062,31,8,2,3,4,1,6,110,-0.063111,-0.044273,2835,58,7,0,4,8,0,16,674,0.204637,0.032833,2799,108,11,2,1,2,20,100,22,-0.193536,-0.090083,6882,126,11,6,0,2,14,75,1715,-0.138627,-0.002165,6510,12,13,6,1,0,10,121,273,0.055823,-0.079495,8103,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,5847415870,2495,1425,5,9,110,128,129,7,70,94,16,52,76,110,11,1,4,10,2,15,339,-0.206514,-0.104366,3279,129,13,5,4,3,2,63,16,-0.159940,-0.346862,6541,76,13,5,4,5,6,89,531,-0.261692,-0.165123,7871,7,12,1,5,3,0,39,112,-0.309379,-0.339529,4052,94,17,1,3,2,12,218,729,-0.295204,0.042390,11144,52,14,3,4,3,6,142,505,0.282166,-0.240743,9490,128,12,2,2,9,2,49,1980,0.115935,-0.137884,5620,16,14,2,5,9,13,122,631,-0.090013,-0.076795,8916,9,11,4,2,6,0,28,893,0.177429,-0.036412,5848,70,19,9,1,2,18,212,1070,-0.036976,0.167983,14725,True,False,False,True,False,False,True,True,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
2,5847405193,2730,1399,5,13,111,85,42,128,7,53,120,73,19,73,18,3,0,7,2,215,654,-0.267973,0.238317,16399,120,16,7,3,5,4,115,1112,-0.393198,-0.317296,8612,85,12,3,8,6,3,16,193,-0.171592,0.172735,3258,13,16,5,2,9,6,145,1787,0.010047,-0.217593,10247,7,12,1,4,6,3,54,1366,-0.051622,-0.067361,5436,19,14,6,4,6,6,116,203,-0.045636,0.338653,8308,111,11,3,4,8,6,24,219,0.061740,0.164986,3984,42,14,1,4,4,5,167,758,-0.012389,0.172875,9253,53,13,6,3,5,21,132,145,0.431347,0.375543,9260,128,11,1,4,6,12,38,567,0.274952,-0.402310,4757,True,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False
3,5847436819,1843,920,5,58,102,31,32,53,108,101,12,126,73,102,6,0,5,4,1,9,353,-0.363910,-0.018785,2143,101,8,4,4,3,0,6,612,-0.303333,0.030635,3117,73,11,0,3,1,6,93,899,-0.286784,0.057870,6999,53,9,3,4,4,11,66,1267,-0.377244,0.010394,5272,32,10,3,1,2,8,43,817,-0.349506,0.021796,4222,31,8,3,4,4,1,6,490,-0.283560,0.036262,3215,58,7,0,4,8,0,16,872,-0.034007,-0.233581,2972,108,12,2,1,3,20,100,368,-0.331358,0.013855,7228,126,12,6,0,3,14,75,2061,-0.355474,0.022363,6856,12,13,6,1,0,10,

In [132]:
raw_data = data.copy()

### Add a column with victory True or False (Radiant)

In [21]:
match_victory = dict((el,0) for el in raw_data.matchid.unique())
def append_victory_column(raw_data):
    for i in raw_data.matchid.unique():
        if match_victory[i] != 0:
            continue
        try: 
            body = requests.get("https://api.steampowered.com/IDOTA2Match_570/GetMatchDetails/v1/?access_token=f5e2a89c137f1564b65504e9d7f3454d&match_id={}&key=03596879227149016F52869E8C8FD226".format(i)).json()
            match_victory[i] = body["result"]['radiant_win']
        except KeyError: 
            continue
        except ConnectionError:
            continue

In [22]:
append_victory_column(raw_data)

In [133]:
match_victory_df = pd.DataFrame.from_dict(match_victory, orient='index',
                       columns=['radiant_victory'])
match_victory_df

,radiant_victory
5847436819,False
5847415870,False
5847405193,True
5847447718,False
5847436254,False
...,...
5906412580,False
5906449790,False
5906447280,True
5906461692,False


In [134]:
match_victory_df = match_victory_df.reset_index()
match_victory_df = match_victory_df.rename(columns={"index": "matchid"})
raw_data = raw_data.merge(match_victory_df, on="matchid")

### Sort values to group it by match id and to have sequential game_time

In [135]:
raw_data = raw_data.sort_values(by=["matchid", "game_time"])

### Slice the data to 15 first mins per each game

In [136]:
start_time = raw_data.groupby(["matchid"]).agg({"game_time": "min"}).reset_index()
bad_start_time = start_time[start_time["game_time"]>100]

In [137]:
raw_data = raw_data[~raw_data["matchid"].isin(bad_start_time.matchid)]

In [138]:
end_time = raw_data.groupby(["matchid"]).agg({"game_time": "max"}).reset_index()
bad_end_time = end_time[end_time["game_time"] < 1000]

In [139]:
raw_data = raw_data[~raw_data["matchid"].isin(bad_end_time.matchid)]

In [140]:
raw_data = raw_data.reset_index()
start_time_indexes = raw_data.groupby(["matchid"]).agg({"index": "min"})
start_time_indexes = start_time_indexes.reset_index()
start_time_indexes

,matchid,index
0,5847436254,85
1,5847447718,44
2,5848236315,260
3,5848238726,314
4,5848244789,339
...,...,...
2120,5906412580,77187
2121,5906447280,77253
2122,5906449790,77211
2123,5906461692,77289


In [141]:
first_fifteen_minutes_indexes = []

for i in start_time_indexes["index"]:   
     for j in range(15):
         first_fifteen_minutes_indexes.append(i + j)
        
first_fifteen_minutes_indexes_array = np.asarray(first_fifteen_minutes_indexes)

In [142]:
raw_data = raw_data[raw_data["index"].isin(first_fifteen_minutes_indexes)]

#### Need to remove examples with other than 15 mins lenght

In [143]:
temp = raw_data.groupby(["matchid"]).agg({"index":"count"})
temp = temp.reset_index()
raw_data = raw_data[~raw_data["matchid"].isin(temp[temp["index"] != 15].matchid)]
raw_data = raw_data.drop(columns="index")

## Normalizing data

#### Replace True and False with 1 and 0

In [146]:
true_and_false_replcer = {True: 1,
                         False: 0,
                         "True": 1,
                         "False": 0,
                         0: 0}
raw_data = raw_data.replace(true_and_false_replcer)

#### Get rid of matchid, timestamp, game_state

In [149]:
raw_data = raw_data.drop(columns=["matchid", "timestamp", "game_state"])

#### Data normalization

In [150]:
bools = ['r_l1_t1', 'r_l1_t2', 'r_l1_t3', 'r_l2_t1', 'r_l2_t2', 'r_l2_t3', 'r_l3_t1', 'r_l3_t2', 'r_l3_t3', 'r_l2_t4_1', 'r_l2_t4_2',\
        'r_l1_b1', 'r_l1_b2', 'r_l2_b1', 'r_l2_b2', 'r_l3_b1', 'r_l3_b2',\
        'd_l1_t1', 'd_l1_t2', 'd_l1_t3', 'd_l2_t1', 'd_l2_t2', 'd_l2_t3', 'd_l3_t1', 'd_l3_t2', 'd_l3_t3', 'd_l2_t4_1', 'd_l2_t4_2',\
        'd_l1_b1', 'd_l1_b2', 'd_l2_b1', 'd_l2_b2', 'd_l3_b1', 'd_l3_b2', 'radiant_victory']


def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        if feature_name not in bools:
            max_value = df[feature_name].max()
            min_value = df[feature_name].min()
            result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
        else:
            continue     
    return result

In [151]:
raw_data = normalize(raw_data)

In [153]:
#saving data to csv file to not lose it after Jupiter reload
norm_data = raw_data
norm_data.to_csv(r"C:\Users\User\Desktop\Python\normalized_data.csv",index=False)

#### separate X and Y

In [154]:
y = norm_data["radiant_victory"]
X = norm_data.drop(["radiant_victory"], axis=1)

In [161]:
X

,game_time,pick_1,pick_2,pick_3,pick_4,pick_5,pick_6,pick_7,pick_8,pick_9,pick_10,r1_hero_id,r1_level,r1_kills,r1_deaths,r1_assists,r1_denies,r1_lh,r1_gold,r1_x,r1_y,r1_net_worth,r2_hero_id,r2_level,r2_kills,r2_deaths,r2_assists,r2_denies,r2_lh,r2_gold,r2_x,r2_y,r2_net_worth,r3_hero_id,r3_level,r3_kills,r3_deaths,r3_assists,r3_denies,r3_lh,r3_gold,r3_x,r3_y,r3_net_worth,r4_hero_id,r4_level,r4_kills,r4_deaths,r4_assists,r4_denies,r4_lh,r4_gold,r4_x,r4_y,r4_net_worth,r5_hero_id,r5_level,r5_kills,r5_deaths,r5_assists,r5_denies,r5_lh,r5_gold,r5_x,r5_y,r5_net_worth,d1_hero_id,d1_level,d1_kills,d1_deaths,d1_assists,d1_denies,d1_lh,d1_gold,d1_x,d1_y,d1_net_worth,d2_hero_id,d2_level,d2_kills,d2_deaths,d2_assists,d2_denies,d2_lh,d2_gold,d2_x,d2_y,d2_net_worth,d3_hero_id,d3_level,d3_kills,d3_deaths,d3_assists,d3_denies,d3_lh,d3_gold,d3_x,d3_y,d3_net_worth,d4_hero_id,d4_level,d4_kills,d4_deaths,d4_assists,d4_denies,d4_lh,d4_gold,d4_x,d4_y,d4_net_worth,d5_hero_id,d5_level,d5_kills,d5_deaths,d5_assists,d5_denies,d5_lh,d5_gold,d5_x,d5_y,d5_net_worth,r_l1_t1,r_l1_t2,r_l1_t3,r_l2_t1,r_l2_t2,r_l2_t3,r_l3_t1,r_l3_t2,r_l3_t3,r_l2_t4_1,r_l2_t4_2,r_l1_b1,r_l1_b2,r_l2_b1,r_l2_b2,r_l3_b1,r_l3_b2,d_l1_t1,d_l1_t2,d_l1_t3,d_l2_t1,d_l2_t2,d_l2_t3,d_l3_t1,d_l3_t2,d_l3_t3,d_l2_t4_1,d_l2_t4_2,d_l1_b1,d_l1_b2,d_l2_b1,d_l2_b2,d_l3_b1,d_l3_b2
0,0.016931,0.826772,0.401575,0.937500,0.539062,0.195312,0.734375,0.007812,0.421875,0.773438,0.570312,0.40625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002883,0.873735,0.198027,0.018660,0.570312,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006738,0.449962,0.445384,0.017176,0.539062,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.026312,0.906024,0.148301,0.022778,0.007812,0.000000,0.000000,0.000000,0.000000,0.000000,0.002571,0.022064,0.130786,0.872110,0.024640,0.195312,0.000000,0.0000,0.000000,0.000000,0.000000,0.002370,0.022674,0.831597,0.176877,0.017160,0.9375,0.000000,0.000000,0.000000,0.000000,0.000000,0.002370,0.021571,0.876538,0.264738,0.016135,0.734375,0.000000,0.000000,0.0,0.000000,0.023810,0.0000,0.028287,0.920650,0.271814,0.017586,0.828125,0.000000,0.0000,0.0,0.000000,0.021739,0.004717,0.070336,0.469820,0.477092,0.019608,0.421875,0.00,0.000000,0.000000,0.000000,0.029412,0.001582,0.024413,0.136221,0.876821,0.021939,0.773438,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.829435,0.246949,0.039382,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.034222,0.826772,0.401575,0.937500,0.539062,0.195312,0.734375,0.007812,0.421875,0.773438,0.570312,0.40625,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.016577,0.795948,0.084636,0.020719,0.570312,0.038462,0.000000,0.000000,0.000000,0.000000,0.002747,0.010855,0.424006,0.436126,0.020918,0.539062,0.000000,0.0,0.0,0.000000,0.051282,0.012788,0.065549,0.828134,0.078122,0.031499,0.007812,0.041667,0.000000,0.000000,0.000000,0.000000,0.005141,0.050257,0.110818,0.813027,0.029371,0.195312,0.000000,0.0000,0.000000,0.000000,0.000000,0.002370,0.005737,0.819259,0.136742,0.020104,0.9375,0.000000,0.000000,0.000000,0.000000,0.000000,0.002370,0.013330,0.898296,0.126538,0.017097,0.734375,0.000000,0.000000,0.0,0.000000,0.047619,0.0075,0.063792,0.873286,0.104726,0.023153,0.828125,0.041667,0.0000,0.0,0.000000,0.043478,0.018868,0.004462,0.467986,0.455115,0.031648,0.421875,0.04,0.000000,0.000000,0.000000,0.058824,0.007911,0.061192,0.118517,0.852823,0.030508,0.773438,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020862,0.873949,0.152585,0.043562,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.057277,0.826772,0.401575,0.937500,0.539062,0.195312,0.734375,0.007812,0.421875,0.773438,0.570312,0.40625,0.043478,0.000000,0.000000,0.000000,0.055556,0.003067,0.007748,0.048539,0.031234,0.022777,0.570312,0.076923,0.000000,0.000000,0.000000,0.000000,0.019231,0.019465,0.489984,0.469072,0.029473,0.539062,0.037037,0.0,0.0,0.000000,0.051282,0.028133,0.006001,0.892825,0.165032,0.042204,0.007812,0.083333,0.000000,0.

In [191]:
test_size = 0.1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=(int((X.shape[0] / 15)*test_size)*15), random_state=42, shuffle=False)

In [192]:
X_train_array = X_train.to_numpy()
X_train_array = X_train_array.reshape(int(X_train_array.shape[0]/15), 15, 155)
X_test_array = X_test.to_numpy()
X_test_array = X_test_array.reshape(int(X_test_array.shape[0]/15), 15, 155)

#### For y we have to create a compatible Series (only one value for 15 rows)

In [193]:
radiant_victory_list_train = []
radiant_victory_list_test = []
i = 0
for index, value in y_train.items():
    i += 1
    if i%15 == 0:
        radiant_victory_list_train.append(value)

i = 0
for index, value in y_test.items():
    i += 1
    if i%15 == 0:
        radiant_victory_list_test.append(value)        

y_train_array = pd.Series(radiant_victory_list_train).to_numpy()
y_test_array = pd.Series(radiant_victory_list_test).to_numpy()

### Building our model

In [194]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(200, activation='relu', return_sequences=True, input_shape=(15, 155)),
    tf.keras.layers.LSTM(100, activation='relu', return_sequences=True),
    tf.keras.layers.LSTM(50, activation='relu', return_sequences=True),
    tf.keras.layers.LSTM(25, activation='relu'),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    
])

# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(500, activation='relu', return_sequences=True, input_shape=(15, 155)),
#     tf.keras.layers.LSTM(400, activation='relu', return_sequences=True),
#     tf.keras.layers.LSTM(300, activation='relu', return_sequences=True),
#     tf.keras.layers.LSTM(200, activation='relu'),
#     tf.keras.layers.Dense(100, activation='relu'),
#     tf.keras.layers.Dense(80, activation='relu'),
#     tf.keras.layers.Dense(50, activation='relu'),
#     tf.keras.layers.Dense(25, activation='relu'),
#     tf.keras.layers.Dense(1, activation='sigmoid')
    
# ])

In [195]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])

In [198]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    min_delta = 0.001,
    patience = 5,
    restore_best_weights = True
)

model.fit(X_train_array, y_train_array, validation_split = 0.2, epochs=40, callbacks=[early_stopping], verbose=2)

Epoch 1/40
48/48 - 9s - loss: 0.1075 - accuracy: 0.9616 - val_loss: 1.8577 - val_accuracy: 0.6878
Epoch 2/40
48/48 - 9s - loss: 0.0579 - accuracy: 0.9762 - val_loss: 1.4951 - val_accuracy: 0.6746
Epoch 3/40
48/48 - 9s - loss: 0.1038 - accuracy: 0.9576 - val_loss: 1.4830 - val_accuracy: 0.6825
Epoch 4/40
48/48 - 9s - loss: 0.0722 - accuracy: 0.9735 - val_loss: 1.4209 - val_accuracy: 0.6905
Epoch 5/40
48/48 - 9s - loss: 0.0773 - accuracy: 0.9735 - val_loss: 1.7728 - val_accuracy: 0.6481
Epoch 6/40
48/48 - 9s - loss: 0.0937 - accuracy: 0.9695 - val_loss: 1.3396 - val_accuracy: 0.6720
Epoch 7/40
48/48 - 10s - loss: 0.0476 - accuracy: 0.9848 - val_loss: 1.9553 - val_accuracy: 0.6852
Epoch 8/40
48/48 - 9s - loss: 0.0519 - accuracy: 0.9801 - val_loss: 2.0343 - val_accuracy: 0.6720
Epoch 9/40
48/48 - 9s - loss: 0.0923 - accuracy: 0.9702 - val_loss: 1.5074 - val_accuracy: 0.6587
Epoch 10/40
48/48 - 9s - loss: 0.0344 - accuracy: 0.9881 - val_loss: 2.6589 - val_accuracy: 0.6958
Epoch 11/40
48/48 